In [36]:
### start_imports ###
import os, sys
from utils import *
from download_DB import download_db
import glob
import pandas as pd
import numpy as np
### end_imports ###
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

### start_miscellaneous ###
from datetime import date,datetime,timedelta
hoje =  pd.to_datetime(date.today()) 

final_do_mes = pd.to_datetime(datetime(hoje.year,hoje.month+1,1) - timedelta(seconds=1))

biu_download_cols  = ['vmonitoramentoleilao', 'vmonitoramentoug', 'vmonitoramentousina', 'vrapeelacesso', 'vrapeelcontratorecurso', 'vrapeelcronograma', 'vrapeelempreendimento', 'vrapeellicenciamento', 'vrapeeloperacaoug']
biu_path = os.path.join(get_standard_folder_path("Documents"),"Previsor/BIU")

def ultimo_rapeel(df):
    df_ultimo_envio = df.loc[df.groupby("IdeUsinaOutorga")['DthEnvio'].idxmax()][['IdeUsinaOutorga','DthEnvio']]
    return pd.merge(df_ultimo_envio,df,on=["IdeUsinaOutorga","DthEnvio"],how="left")


list_carac_usina = ['Usina_Sem_Rapeel','CondicaoAmbiental','Homologar_Marcos','Revisar_IO','Usina_Selecionada','Paralisada','Sem_Previsao','criterionovopmo','Prev_OC_passado','Em_teste','Sem_Monitoramento','Manual','UG_Sem_Rapeel']
list_carac_ug = ['FASE','flagOPTeste30dias','UG_Sem_Rapeel','criterionovopmo']
list_casos = ['Caso_I','Caso_II_a','Caso_II_b','Caso_III']
list_ambiental = ['DatValidadeLI','DatValidadeLP','DatValidadeLO','ValidadeAmbiental','CondicaoAmbiental']
list_contratos = ['NomSitContratoCCD', 'NomSitContratoCUST', 'NomSituacaoContratoCUSD', 'NomSitContratoCCT']
### end_miscellaneous ###

In [37]:
#### start_function ### function_name = generate_BIU
### start_function_vars ###
### start_funcion_defined_vars ###
### end_funcion_defined_vars ###
biu_download_path = os.path.join(biu_path,"Download")
previsao_file = os.path.join(biu_path,"Estagio_I/Previsao_OC.gzip")
### end_function_vars ###

In [38]:
# Lista de colunas que serão usadas várias vezes para a realização de merges entre os dataframes
list_id_data = ['DthEnvio','IdeUsinaOutorga']
list_id_ug = ['IdeUsinaOutorga','NumOperacaoUg']

# Informações de Usinas

## Carrega banco de dados

In [39]:
cols = ["IdeUsinaOutorga",'NomUsina','CodCegFormatado','DatMonitoramento',"DatCanteiroObraRealizado","DatDesvioRioRealizado","DatEnchimentoRealizado","DatConclusaoSisTransRealizado","DatPrevisaoIniciobra","IdcObraParalisada","IdcUsinaMonitorada","DatConcretagemRealizado","DatConclusaoTorresRealizado","DatInicioObraOutorgado","DatSisTransmissaoRealizado","IdcSituacaoObra","IdcSemPrevisao","DatComissionamentoUGRealizado","SigTipoGeracao","DscComercializacaoEnergia","DatInicioObraRealizado","DatMontagemRealizado",'DscJustificativaPrevisao']
# Carrega informações do monitoramento
df_usina = pd.read_parquet(os.path.join(biu_download_path,"vmonitoramentousina.gzip"))[cols]

#Transforma coluna em booleano
df_usina.IdcUsinaMonitorada = df_usina.IdcUsinaMonitorada.map({"Sim":"True","Não":False}).astype(bool)

# Remove usinas que não são monitoradas
df_usina = df_usina[df_usina.IdcUsinaMonitorada].reset_index(drop=True)

In [40]:
# Excluir ? Não está sendo usado
monitoramentoleilao  = pd.read_parquet(os.path.join(biu_download_path,"vmonitoramentoleilao.gzip"))[['IdeUsinaOutorga','CodLeilao', 'DatInicioSuprimento']]
monitoramentoleilao.dropna(subset='DatInicioSuprimento',inplace=True)
# Mantém apenas primeira data de suprimento 
monitoramentoleilao = monitoramentoleilao.loc[monitoramentoleilao.groupby('IdeUsinaOutorga').DatInicioSuprimento.idxmin()]
df_usina = pd.merge(df_usina,monitoramentoleilao,how="left",on="IdeUsinaOutorga")

In [41]:
cols = ['IdeUsinaOutorga','CodCeg','DthEnvio','DatRealizacaoII','DatPrevistaAprovacaoIII','DatRealizacaoIII','DatRealizacaoIX','DatRealizacaoXI','DatRealizacaoXIII','DatRealizacaoXIV','DatRealizacaoXII','DatRealizacaoVII','DatRealizacaoVIII','DatRealizacaoVI','DatRealizacaoX','DatRealizacaoV','DatRealizacaoIV']
rename_cols = {
        'DatRealizacaoII' : 'canteiroReal',
        'DatPrevistaAprovacaoIII' : 'prev_IO',
        'DatRealizacaoIII' : 'IO_real',
        'DatRealizacaoIX' : 'DesvRio_real',
        'DatRealizacaoXI' : 'Ench_Real',
        'DatRealizacaoXIII' : 'IOTrans_Real',
        'DatRealizacaoXIV' : 'Conc_Trans_REAL',
        'DatRealizacaoXII' : 'Comiss_Real',
        'DatRealizacaoVII' : 'ME_Real_conc_eol',
        'DatRealizacaoVIII' : 'Conc_Combust_Real'}
        
rapeel_cronograma = pd.read_parquet(os.path.join(biu_download_path,"vrapeelcronograma.gzip"))[cols].rename(columns=rename_cols)
rapeel_cronograma = rapeel_cronograma.loc[rapeel_cronograma.groupby('IdeUsinaOutorga').DthEnvio.idxmax()]

# Faz merge left entre as informações do monitoramento e do rapeel
df_usina = pd.merge(df_usina,rapeel_cronograma,how="left",on="IdeUsinaOutorga")

In [42]:
# Cria coluna para identificar usinas que não enviaram rapeel
df_usina["Usina_Sem_Rapeel"] = True
df_usina.loc[df_usina.IdeUsinaOutorga.isin(rapeel_cronograma.IdeUsinaOutorga),'Usina_Sem_Rapeel'] = False

In [43]:
cols = ['DthEnvio','IdeUsinaOutorga','DatConclusaoIII',]
rename_cols = {'DatConclusaoIII':'DatConclusaoACL'}
mercado = pd.read_parquet(os.path.join(biu_download_path,"vrapeelcontratorecurso.gzip"))[cols].rename(columns=rename_cols)
mercado = mercado.loc[mercado.groupby("IdeUsinaOutorga").DthEnvio.idxmax()]
del mercado['DthEnvio']
df_usina = pd.merge(df_usina,mercado,on="IdeUsinaOutorga",how="left")

In [44]:
cols = ["DatValidadeIV","DatValidadeI","DthEnvio","IdeUsinaOutorga","DatValidadeV"]
rename_cols = {'DatValidadeIV': 'DatValidadeLI','DatValidadeI': 'DatValidadeLP','DatValidadeV': 'DatValidadeLO'}
ambiental_datas = pd.read_parquet(os.path.join(biu_download_path,"vrapeellicenciamento.gzip"))[cols].rename(columns=rename_cols)
ambiental_datas = ambiental_datas.loc[ambiental_datas.groupby('IdeUsinaOutorga').DthEnvio.idxmax()]
del ambiental_datas['DthEnvio']
df_usina = pd.merge(df_usina,ambiental_datas,on="IdeUsinaOutorga",how="left")

In [45]:
cols = ["DthEnvio","NomSitContratoI","NomSitContratoIV","NomSituacaoContratoIII","NomSitContratoII","IdeUsinaOutorga"]
rename_cols = {'NomSitContratoI': 'NomSitContratoCCD','NomSitContratoIV': 'NomSitContratoCUST','NomSituacaoContratoIII': 'NomSituacaoContratoCUSD','NomSitContratoII': 'NomSitContratoCCT'}
acesso_contratos = pd.read_parquet(os.path.join(biu_download_path,"vrapeelacesso.gzip"))[cols].rename(columns=rename_cols)
acesso_contratos = acesso_contratos.loc[acesso_contratos.groupby('IdeUsinaOutorga').DthEnvio.idxmax()]
del acesso_contratos['DthEnvio']
df_usina = pd.merge(df_usina,acesso_contratos,on="IdeUsinaOutorga",how="left")

##  Cria colunas com definições e regras

In [46]:
df_usina["classe"] = df_usina.CodCegFormatado.str.slice(4,6)
df_usina["CC_real"] = pd.to_datetime(pd.NA)
df_usina.loc[df_usina.classe == "PH","CC_real"] = df_usina.DatRealizacaoIV
df_usina.loc[df_usina.classe == "CV","CC_real"] = df_usina.DatRealizacaoV
df_usina["ME_real"] = df_usina.DatRealizacaoX
df_usina.loc[df_usina.classe == "CV","ME_real"] = df_usina.DatRealizacaoVI

In [47]:
dict_validades = {
    'DatValidadeLO' : 'LO',
    'DatValidadeLI' : 'LI',
    'DatValidadeLP' : "LP"
}

df_usina['ValidadeAmbiental'] = pd.NA
for validade in dict_validades:
    df_usina.loc[df_usina.ValidadeAmbiental.isna(),'ValidadeAmbiental'] = df_usina[validade]
display(df_usina[['IdeUsinaOutorga'] +list(dict_validades.keys()) + ['ValidadeAmbiental']].sample(5))

,IdeUsinaOutorga,DatValidadeLO,DatValidadeLI,DatValidadeLP,ValidadeAmbiental
844,44566,NaT,NaT,2024-04-25,2024-04-25
798,44453,NaT,NaT,2024-01-03,2024-01-03
1407,49144,NaT,NaT,2025-04-04,2025-04-04
608,38349,NaT,2025-02-16,2022-01-26,2025-02-16
1133,47281,2030-07-02,2030-07-02,2030-07-02,2030-07-02


In [48]:
df_usina['CondicaoAmbiental'] = pd.NA

for validade in reversed(dict_validades):
    df_usina.loc[df_usina[validade].notna(),'CondicaoAmbiental'] = dict_validades[validade]

df_usina.loc[(df_usina.CondicaoAmbiental.isna()) & (df_usina.DthEnvio.notna()),'CondicaoAmbiental'] = 'Sem LP'
df_usina.loc[(df_usina.CondicaoAmbiental.isna()),'CondicaoAmbiental'] = 'Não Informado'


df_usina[list(dict_validades) + ['ValidadeAmbiental','CondicaoAmbiental']].sample(10)

display(df_usina[list(dict_validades.keys()) + ['CondicaoAmbiental','ValidadeAmbiental']])

df_usina['CondicaoConexao'] = "Não informado"

nsa = "Não se Aplica"
nass = "Não Assinado"
valido = "Válido"
vencido = "Vencido"

df_usina.loc[
    ((df_usina.NomSitContratoCCD ==  nsa)&
    (df_usina.NomSitContratoCCT == nsa) &
    df_usina.DthEnvio.notna()
    )
    ,'CondicaoConexao'] = "Verificar"

df_usina.loc[
(    ((df_usina.NomSitContratoCCD ==  valido) |
    (df_usina.NomSitContratoCCT == valido)) &
    df_usina.DthEnvio.notna())
    
    ,'CondicaoConexao'] = "OK"

df_usina.loc[
    (((df_usina.NomSitContratoCCD ==  vencido) |
    (df_usina.NomSitContratoCCT == vencido) |
    (df_usina.NomSitContratoCCD == nass) |
    (df_usina.NomSitContratoCCT == nass )) &
    df_usina.DthEnvio.notna())
    ,'CondicaoConexao'] = "Sem Conexão"
display(df_usina[['IdeUsinaOutorga'] + ['NomSitContratoCCD','NomSitContratoCCT','DthEnvio','CondicaoConexao']].sample(10))


df_usina['CondicaoUso'] = "Não informado"

df_usina.loc[
    ((df_usina.NomSituacaoContratoCUSD ==  nsa)&
    (df_usina.NomSitContratoCUST == nsa) 
    )
    ,'CondicaoUso'] = "Verificar"


df_usina.loc[
    ((df_usina.NomSituacaoContratoCUSD ==  valido) |
    (df_usina.NomSitContratoCUST == valido) 
    )
    ,'CondicaoUso'] = "OK"

df_usina.loc[
    (((df_usina.NomSituacaoContratoCUSD ==  vencido) |
    (df_usina.NomSitContratoCUST == vencido) |
    (df_usina.NomSituacaoContratoCUSD == nass) |
    (df_usina.NomSitContratoCUST == nass )))
    ,'CondicaoUso'] = "Sem Uso"

display(df_usina[['IdeUsinaOutorga'] + ['NomSituacaoContratoCUSD','NomSitContratoCUST','CondicaoConexao']].sample(10))


df_usina["PPA"] = "Ambos"

df_usina.loc[
    (((df_usina.DscComercializacaoEnergia == "ACR") & 
        df_usina.DatConclusaoACL.isna())),
    "PPA"] = "ACR"

df_usina.loc[
    (((df_usina.DscComercializacaoEnergia == "Fora do ACR") & 
        df_usina.DatConclusaoACL.isna())),
    "PPA"] = "Nenhum"

df_usina.loc[
    (((df_usina.DscComercializacaoEnergia == "Fora do ACR") & 
        df_usina.DatConclusaoACL.notna())),
    "PPA"] = "ACL"

display(df_usina[['IdeUsinaOutorga'] + ['DscComercializacaoEnergia','DatConclusaoACL','PPA']].sample(10))

,DatValidadeLO,DatValidadeLI,DatValidadeLP,CondicaoAmbiental,ValidadeAmbiental
0,NaT,2026-03-09,2002-12-11,LI,2026-03-09
1,NaT,NaT,NaT,Sem LP,NaT
2,2026-02-22,2028-06-28,2000-03-29,LO,2026-02-22
3,NaT,NaT,NaT,Não Informado,NaT
4,NaT,2009-03-10,1999-01-09,LI,2009-03-10
...,...,...,...,...,...
2472,NaT,NaT,NaT,Não Informado,NaT
2473,NaT,NaT,NaT,Não Informado,NaT
2474,NaT,NaT,NaT,Não Informado,NaT
2475,NaT,NaT,NaT,Não Informado,NaT


,IdeUsinaOutorga,NomSitContratoCCD,NomSitContratoCCT,DthEnvio,CondicaoConexao
2106,52282,Não se Aplica,Não Assinado,2023-01-03 14:21:06.477,Sem Conexão
1134,47282,Não se Aplica,Válido,2023-01-04 01:54:58.647,OK
1068,47052,Não se Aplica,Não Assinado,2023-01-05 17:01:52.087,Sem Conexão
965,46533,Não se Aplica,Não Assinado,2023-01-10 16:06:08.210,Sem Conexão
1446,49232,NaN,NaN,NaT,Não informado
2098,52267,Não se Aplica,Não Assinado,2023-01-05 22:18:32.823,Sem Conexão
1824,50466,Não se Aplica,Válido,2023-01-05 16:48:03.717,OK
2090,52259,Não se Aplica,Não Assinado,2023-01-03 14:52:53.587,Sem Conexão
2363,55334,NaN,NaN,NaT,Não informado
2295,54648,Não se Aplica,Não Assinado,2023-01-02 08:26:18.240,Sem Conexão


,IdeUsinaOutorga,NomSituacaoContratoCUSD,NomSitContratoCUST,CondicaoConexao
1740,49939,Não se Aplica,Não Assinado,Sem Conexão
2283,54636,Não se Aplica,Não Assinado,Sem Conexão
16,27982,Não se Aplica,Não se Aplica,Verificar
2387,55386,NaN,NaN,Não informado
88,31768,Não Assinado,Não se Aplica,Sem Conexão
1752,49962,Não se Aplica,Não se Aplica,Verificar
2049,51983,Não se Aplica,Não Assinado,Sem Conexão
2302,54659,Não se Aplica,Não se Aplica,Verificar
645,40696,Não se Aplica,Válido,OK
68,30675,Válido,Não se Aplica,OK


,IdeUsinaOutorga,DscComercializacaoEnergia,DatConclusaoACL,PPA
2087,52244,Fora do ACR,NaT,Nenhum
2048,51873,Fora do ACR,NaT,Nenhum
1311,48574,Fora do ACR,NaT,Nenhum
882,44992,Fora do ACR,NaT,Nenhum
1761,50012,Fora do ACR,NaT,Nenhum
2258,54553,Fora do ACR,NaT,Nenhum
1220,47383,Fora do ACR,NaT,Nenhum
602,38284,Fora do ACR,NaT,Nenhum
514,37893,Fora do ACR,NaT,Nenhum
585,38135,Fora do ACR,NaT,Nenhum


In [49]:
ea = "Em andamento"
na = "Não Iniciada"
nenhum = "Nenhum"
paralisada = "Paralisada"
ok = "OK"

In [50]:
slp = "Sem LP"
sim = "Sim"
df_usina['criterio_novo'] = np.select(
    [
        (df_usina.IdcSemPrevisao == sim),
        (df_usina.IdcSituacaoObra == ea),
        (( df_usina.IdcSituacaoObra == na)  &  (df_usina.PPA != nenhum) ),
        ((df_usina.IdcSituacaoObra == na) & (df_usina.PPA == nenhum) & (df_usina.CondicaoAmbiental.isin(["LI","LO"])) & (df_usina.CondicaoUso == ok)),
        ((df_usina.IdcSituacaoObra == na) & (df_usina.PPA == nenhum) & (df_usina.CondicaoAmbiental == "LP") & (df_usina.CondicaoUso == ok)),
        ((df_usina.IdcSituacaoObra == na) & (df_usina.PPA == nenhum) & (df_usina.CondicaoAmbiental.isin(["LI","LO"])) & (df_usina.CondicaoUso != ok)),
        ((df_usina.IdcSituacaoObra == na) & (df_usina.PPA == nenhum) & (df_usina.CondicaoAmbiental == "LP") & (df_usina.CondicaoUso != ok)),
        (df_usina.IdcSituacaoObra == paralisada),
        (df_usina.CondicaoAmbiental == "Sem LP"),
        ((df_usina.DthEnvio.isna()) & (df_usina.PPA == nenhum)),
        
    ],
    [9,0,1,2,4,3,5,8,6,7],
    default=-8
)


display(df_usina[['IdeUsinaOutorga','IdcSituacaoObra','PPA','CondicaoAmbiental','CondicaoUso','criterio_novo']].sample(5))

,IdeUsinaOutorga,IdcSituacaoObra,PPA,CondicaoAmbiental,CondicaoUso,criterio_novo
821,44516,Não Iniciada,Nenhum,LI,Sem Uso,3
2138,52389,Não Iniciada,Nenhum,LP,Sem Uso,5
2121,52328,Não Iniciada,Nenhum,LO,OK,2
1306,48542,Não Iniciada,Nenhum,LO,OK,2
1002,46720,Não Iniciada,Nenhum,LI,OK,2


In [51]:
dict_marcos_homologar = {
    'DatInicioObraRealizado' : 'IO_real',
    'DatConcretagemRealizado' : 'CC_real',
    'DatMontagemRealizado' : 'ME_real',
    'DatEnchimentoRealizado' : 'Ench_Real',
    'DatSisTransmissaoRealizado' : 'IOTrans_Real',
    'DatCanteiroObraRealizado' : 'canteiroReal',
    'DatComissionamentoUGRealizado' : 'Comiss_Real',
    'DatConclusaoSisTransRealizado' : 'Conc_Trans_REAL',
    'DatDesvioRioRealizado' : 'DesvRio_real',
    'DatConclusaoTorresRealizado' : 'ME_Real_conc_eol'}
    
df_usina['Homologar_Marcos'] = False
df_usina['Dsc_Marcos_a_Homologar'] = ''

# Homologação de marcos

for marco_monitoramento, marco_rapeel in dict_marcos_homologar.items():
    mask = (df_usina[marco_monitoramento].isna()) & (df_usina[marco_rapeel].notna())
    df_usina['Homologar_Marcos'] |= mask
    df_usina.loc[mask,'Dsc_Marcos_a_Homologar'] += f'{marco_monitoramento}, '

df_usina.Dsc_Marcos_a_Homologar = df_usina.Dsc_Marcos_a_Homologar.str.slice(0,-2)
display(df_usina[df_usina.Homologar_Marcos][["IdeUsinaOutorga",'NomUsina'] + ['Homologar_Marcos','Dsc_Marcos_a_Homologar']].sample(5))

,IdeUsinaOutorga,NomUsina,Homologar_Marcos,Dsc_Marcos_a_Homologar
660,40727,Belmonte 1-3,True,DatSisTransmissaoRealizado
1244,47424,Serra do Mel III (Antiga Serra do Mel IX),True,DatInicioObraRealizado
1511,49416,Ventos de Santa Luzia 03,True,DatSisTransmissaoRealizado
1500,49399,Ventos de Santa Luzia 02,True,DatSisTransmissaoRealizado
1884,50842,Arinos 23,True,DatCanteiroObraRealizado


In [52]:
df_usina['Revisar_IO'] =  np.select(
 [(df_usina.DatPrevisaoIniciobra < (hoje + pd.Timedelta(15,"D"))) |
 (df_usina.prev_IO > df_usina.DatPrevisaoIniciobra)],
 [True],
 default = False)
display(df_usina[df_usina.Revisar_IO][['IdeUsinaOutorga', 'NomUsina','DatPrevisaoIniciobra']].sample(5))

,IdeUsinaOutorga,NomUsina,DatPrevisaoIniciobra
623,38389,Panatí 1,2023-02-01
1040,46882,Boca do Riacho 17,2023-02-15
654,40708,Gameleira 1,2023-02-01
630,38397,Sitiá 2,2023-02-01
280,35147,Clairto Zonta,2023-02-15


In [53]:
usinas_selecionadas = read_file("./usinas_selecionadas.txt").split(",")
usinas_selecionadas = [int(num) for num in usinas_selecionadas]
df_usina["Usina_Selecionada"] = False
df_usina.loc[df_usina.IdeUsinaOutorga.isin(usinas_selecionadas),'Usina_Selecionada'] = True

df_usina['Paralisada'] = False
df_usina.loc[df_usina.IdcObraParalisada=="Sim",'Paralisada'] = True

df_usina['Sem_Previsao'] = False
df_usina.loc[df_usina.IdcSemPrevisao=="Sim",'Sem_Previsao'] = True

display(df_usina.loc[df_usina.Usina_Selecionada & df_usina.Paralisada,['IdeUsinaOutorga', 'NomUsina','Usina_Selecionada','Paralisada','Sem_Previsao']])

,IdeUsinaOutorga,NomUsina,Usina_Selecionada,Paralisada,Sem_Previsao
53,30150,Almirante Álvaro Alberto - Unidade III (Antiga...,True,True,True
2389,55592,Edlux X,True,True,True
2392,55739,EPP II,True,True,True
2393,55740,EPP IV,True,True,True
2394,55856,Rio de Janeiro I,True,True,True
2395,56139,RE TG 100 02 01,True,True,True


# Informações de UGs

## Carrega banco de dados

In [54]:
cols = ["IdeUsinaOutorga",'NumUgUsina',"IdcMonitorada",'DatLiberOpComerRealizado']
monitoramentoug = pd.read_parquet(os.path.join(biu_download_path,"vmonitoramentoug.gzip"))[cols].rename(columns={'NumUgUsina':'NumOperacaoUg'})
# Não remover ugs não monitoradas nesse passo, pois serão usadas para encontrar ugs difentes entre 
# monitoramento e rapeel. São retiradas durante o merge
monitoramentoug.IdcMonitorada = monitoramentoug.IdcMonitorada.map({"Sim":True,"Não":False}).astype(bool)
display(monitoramentoug.sample(5))

,IdeUsinaOutorga,NumOperacaoUg,IdcMonitorada,DatLiberOpComerRealizado
21486,40725,7,True,NaT
78805,54778,212,True,NaT
31408,40861,22,False,2022-10-04
29244,40725,27,True,NaT
77167,53887,20,True,NaT


In [55]:
cols = ["IdeUsinaOutorga",'NumOperacaoUg',"DthEnvio",'DatPrevistaComercial']
ug_rapeel = pd.read_parquet(os.path.join(biu_download_path,"vrapeeloperacaoug.gzip"))[cols]
ug_rapeel = ultimo_rapeel(ug_rapeel)
display(ug_rapeel.head(5))

,IdeUsinaOutorga,DthEnvio,NumOperacaoUg,DatPrevistaComercial
0,273,2022-10-09 21:04:17.743,4,2023-11-02
1,273,2022-10-09 21:04:17.743,3,2023-10-03
2,324,2023-01-02 08:34:07.347,1,2026-07-01
3,601,2023-01-03 19:22:43.140,1,2027-05-01
4,601,2023-01-03 19:22:43.140,3,2027-05-01


In [56]:
cols_used_robot = ['DatInicioObraOutorgado','DatPrevisaoIniciobra','prev_IO']
list_cols_used =   ['IdeUsinaOutorga','IdcObraParalisada','IdcSemPrevisao','criterio_novo','Usina_Sem_Rapeel',] + cols_used_robot
# UGs não monitoradas são retiradas aqui
df_ug = pd.merge(df_usina[list_cols_used],monitoramentoug[monitoramentoug.IdcMonitorada],on="IdeUsinaOutorga",how='left')
df_ug.IdcMonitorada = df_ug.IdcMonitorada.astype(bool)
df_ug = pd.merge(df_ug,ug_rapeel,on=list_id_ug,how='left') # poderia ser inner?
# Remove usinas que já entraram em operação comercial ou usinas sem UGs monitoradas
df_ug = df_ug.loc[df_ug.DatLiberOpComerRealizado.isna() &  df_ug.NumOperacaoUg.notna()].reset_index(drop=True)

In [57]:
calculo_previsao = pd.read_parquet(previsao_file)
calculo_previsao.FaseAtual =calculo_previsao.FaseAtual.str.slice(0,-3)
calculo_previsao.Indicador =calculo_previsao.Indicador/100

cols = ["Dat_OC_obrigacao","DatPrevisaoIniciobra","FASE","Ind_crono_norm","IdeUsinaOutorga","flagOPTeste30dias",'Previsao_OC']
rename_cols = {'FaseAtual':'FASE','Indicador':'Ind_crono_norm','NumUgUsina':'NumOperacaoUg'}
calculo_previsao = calculo_previsao.rename(columns=rename_cols)[['IdeUsinaOutorga','NumOperacaoUg','FASE','Ind_crono_norm','flagOPTeste30dias','Previsao_OC','Dat_OC_obrigacao','MdaPotenciaUnitaria']] 

df_ug = pd.merge(df_ug,calculo_previsao,on=list_id_ug,how="left")

display(df_ug.head(5))

,IdeUsinaOutorga,IdcObraParalisada,IdcSemPrevisao,criterio_novo,Usina_Sem_Rapeel,DatInicioObraOutorgado,DatPrevisaoIniciobra,prev_IO,NumOperacaoUg,IdcMonitorada,DatLiberOpComerRealizado,DthEnvio,DatPrevistaComercial,FASE,Ind_crono_norm,flagOPTeste30dias,Previsao_OC,Dat_OC_obrigacao,MdaPotenciaUnitaria
0,324,Não,Sim,9,False,2014-02-28,NaT,2025-02-01,1.0,True,NaT,2023-01-02 08:34:07.347,2026-07-01,OUT,3.312128e-12,0,2023-10-20 05:07:04.944,2015-06-30,7500.0
1,601,Não,Sim,9,False,2001-01-30,NaT,2026-05-29,3.0,True,NaT,2023-01-03 19:22:43.140,2027-05-01,OUT,6.749046e-13,0,2025-08-13 12:20:37.555,2003-10-31,5000.0
2,601,Não,Sim,9,False,2001-01-30,NaT,2026-05-29,1.0,True,NaT,2023-01-03 19:22:43.140,2027-05-01,OUT,6.749046e-13,0,2025-08-13 12:20:37.555,2003-10-31,5000.0
3,601,Não,Sim,9,False,2001-01-30,NaT,2026-05-29,2.0,True,NaT,2023-01-03 19:22:43.140,2027-05-01,OUT,6.749046e-13,0,2025-08-13 12:20:37.555,2003-10-31,5000.0
4,659,Sim,Sim,9,False,2002-04-30,NaT,2023-02-10,4.0,True,NaT,2023-01-06 09:37:52.703,2023-06-12,ME,5.193540e-01,0,2023-04-24 22:27:14.092,2021-01-01,648.0


In [58]:
monitoramentoug['Existe_Monitoramento'] = True     # Adiciona colunas para checagem
ug_rapeel['Existe_Rapeel'] = True                  # dos resultados

# Faz merge tipo outer (todas as UGs) com os dados do  
# monitoramento e rapeel

comp_ug = pd.merge(monitoramentoug[list_id_ug + ['Existe_Monitoramento',"IdcMonitorada"]], 
        ug_rapeel[list_id_ug + ['Existe_Rapeel']], how="outer")                            

# Inner join com usinas monitoradas
comp_ug = pd.merge(df_usina[df_usina.IdcUsinaMonitorada][['IdeUsinaOutorga']],comp_ug,how="inner",on="IdeUsinaOutorga")

# Inner join com usinas que já enviaram rapeel
comp_ug = pd.merge(rapeel_cronograma[['IdeUsinaOutorga']],comp_ug,how="inner",on="IdeUsinaOutorga") 

# Das UGs que estão no monitoramento, remove-se UGs não monitoradas
comp_ug = comp_ug[comp_ug.IdcMonitorada != False]

# Preenche com o valor falso as usinas que não foram encontradas
comp_ug[['Existe_Monitoramento','Existe_Rapeel']] = comp_ug[['Existe_Monitoramento','Existe_Rapeel']].fillna(False) 

# Coluna com diferenças
comp_ug["Diff"] = ~(comp_ug.Existe_Monitoramento & comp_ug.Existe_Rapeel) 


df_UG_Sem_Rapeel = comp_ug[comp_ug.Existe_Monitoramento & comp_ug.Diff][['IdeUsinaOutorga']].drop_duplicates()
df_UG_Sem_Rapeel['UG_Sem_Rapeel'] = True


# Todas as UGs de uma usina que pelo menos uma UG não possua rapeel terão UG_Sem_Rapeel = True
df_ug = pd.merge(df_ug,df_UG_Sem_Rapeel,on='IdeUsinaOutorga',how='left')
df_ug.loc[df_ug.UG_Sem_Rapeel.isna(),'UG_Sem_Rapeel'] = False


df_ug = df_ug.merge(ug_rapeel.loc[ug_rapeel[ug_rapeel.DatPrevistaComercial.notna()].groupby("IdeUsinaOutorga").DatPrevistaComercial.idxmax()][['IdeUsinaOutorga','DatPrevistaComercial']].rename(columns={'DatPrevistaComercial':'PrevisaoOC_rapeel_max'}),
            how="left",on="IdeUsinaOutorga")


# Para esses casos, em vez de DatPrevistaComercial, será usado PrevisaoOC_rapeel_max
# Ver arquivo 
df_ug.loc[df_ug.UG_Sem_Rapeel | df_ug.DatPrevistaComercial.isna(),'DatPrevistaComercial'] = df_ug.PrevisaoOC_rapeel_max


##  Cria colunas com definições e regras

In [59]:
################################ criterionovopmo ################################
list_condicoes = [
    ((df_ug['criterio_novo'] == 0) & (df_ug['FASE'] != "OT")),
    ((df_ug['criterio_novo'] == 0) & (df_ug['FASE'] == "OT") & (df_ug['flagOPTeste30dias'] == 1)),
    ((df_ug['criterio_novo'] == 0) & (df_ug['FASE'] == "OT")),
    ((df_ug['criterio_novo'] == 1)),
    ((df_ug['criterio_novo'] == 2)),
    ((df_ug['criterio_novo'] == 3)),
    ((df_ug['criterio_novo'] == 5)),
    ((df_ug['criterio_novo'] == 4)),
    ((df_ug['criterio_novo'] == 6)),
    ((df_ug['criterio_novo'] == 7)),
    ((df_ug['criterio_novo'] == 8)),
    ((df_ug['criterio_novo'] == 9)),
]

list_values = [0.1, 0.2, 0.3, 1, 2, 3, 5, 4, 6, 7, 8, 9]

df_ug['criterionovopmo'] = np.select(list_condicoes,list_values)

################################ regranovapmo ################################

df_ug.loc[(df_ug['criterio_novo'] == 0) & (df_ug['FASE'] != "OT"),'regranovapmo'] = df_ug[['DatPrevistaComercial','Previsao_OC']].max(axis=1)


df_ug.loc[(df_ug['criterio_novo'] == 0) & (df_ug['FASE'] == "OT") & (df_ug['flagOPTeste30dias'] == 1),'regranovapmo'] = ( hoje + pd.Timedelta(60,"D"))

df_ug.loc[(df_ug['criterio_novo'] == 0) & (df_ug['FASE'] == "OT") & (df_ug['Previsao_OC'].isna()),'regranovapmo'] = df_ug['DatPrevistaComercial']


df_ug.loc[(df_ug['criterio_novo'] == 0) & (df_ug['FASE'] == "OT") & (df_ug['Previsao_OC'].notna()),'regranovapmo'] = df_ug['Previsao_OC']

df_ug.loc[(df_ug['criterio_novo']== 1) | (df_ug['criterio_novo']== 2) ,'regranovapmo'] = df_ug[['DatPrevistaComercial','Previsao_OC','Dat_OC_obrigacao']].max(axis=1)
df_ug["Dummy"] = hoje + pd.Timedelta(5*365,'D')

df_ug.loc[((df_ug['criterio_novo'] == 3) | (df_ug['criterio_novo'] == 4) | (df_ug['criterio_novo'] == 5) ),'regranovapmo'] = df_ug[['DatPrevistaComercial','Previsao_OC','Dat_OC_obrigacao','Dummy']].max(axis=1)


df_ug.loc[df_ug['criterio_novo']== 6,'regranovapmo'] = pd.NA

df_ug["Dummy"] = hoje + pd.Timedelta(6*365,'D')
df_ug.loc[df_ug['criterio_novo']== 7,'regranovapmo'] =  df_ug[['DatPrevistaComercial','Previsao_OC','Dat_OC_obrigacao','Dummy']].max(axis=1)


df_ug.loc[df_ug['criterio_novo']== 8,'regranovapmo'] = df_ug[['DatPrevistaComercial','Dat_OC_obrigacao']].max(axis=1)

df_ug.loc[(df_ug['criterio_novo'] == 9),'regranovapmo'] = pd.NA


In [60]:
serie_usinas_previsao_OC_passado = df_ug[(df_ug.Previsao_OC < final_do_mes) | (df_ug.Previsao_OC.isna())].IdeUsinaOutorga.drop_duplicates()
serie_usinas_previsao_em_teste = df_ug[df_ug.criterionovopmo.isin([0.2,0.3])].IdeUsinaOutorga.drop_duplicates()

serie_flagOPTeste30dias = df_ug[df_ug.flagOPTeste30dias == 1].IdeUsinaOutorga.drop_duplicates()
df_usina["flagOPTeste30dias"] = False
df_usina.loc[df_usina.IdeUsinaOutorga.isin(serie_flagOPTeste30dias),'flagOPTeste30dias'] = True

serie_UG_Sem_Rapeel = df_ug[df_ug.UG_Sem_Rapeel].IdeUsinaOutorga.drop_duplicates()

In [61]:
df_usina["UG_Sem_Rapeel"] = False
df_usina.loc[df_usina.IdeUsinaOutorga.isin(serie_UG_Sem_Rapeel),'UG_Sem_Rapeel'] = True

df_usina["flagOPTeste30dias"] = False
df_usina.loc[df_usina.IdeUsinaOutorga.isin(serie_flagOPTeste30dias),'flagOPTeste30dias'] = True

df_usina["Prev_OC_passado"] = False
df_usina.loc[df_usina.IdeUsinaOutorga.isin(serie_usinas_previsao_OC_passado),'Prev_OC_passado'] = True

df_usina["Em_teste"] = False
df_usina.loc[df_usina.IdeUsinaOutorga.isin(serie_usinas_previsao_em_teste),'Em_teste'] = True

df_usina["Sem_Monitoramento"] = False
df_usina.loc[df_usina.DatMonitoramento < (hoje - pd.Timedelta(4*30, unit="D")),'Sem_Monitoramento'] = True

In [62]:
df_usina["Manual"] = False
df_usina.loc[(df_usina.Paralisada | df_usina.Sem_Previsao | 
    df_usina.Usina_Selecionada| df_usina.Prev_OC_passado | df_usina.Em_teste | 
    (df_usina.DscJustificativaPrevisao == 'Situação das obras de conexão e linha de transmissão associada.')),
    'Manual'] = True

In [63]:
df_usina['Caso_I'] = df_usina['Caso_II_a'] = df_usina['Caso_II_b'] = df_usina['Caso_III'] = False
list_casos = ['Caso_I','Caso_II_a','Caso_II_b','Caso_III']

In [64]:
df_usina['Caso_I'] = (~df_usina.Manual) & (~df_usina.Revisar_IO) & (~ df_usina.Homologar_Marcos) 

df_usina['Caso_II_a'] = (~df_usina.Manual) & (df_usina.Revisar_IO) & (~ df_usina.Homologar_Marcos)

df_usina['Caso_II_b'] = (~df_usina.Manual)  & ((df_usina.Homologar_Marcos) | (df_usina.Homologar_Marcos & df_usina.Revisar_IO))

df_usina['Caso_III'] = (df_usina.Manual & (df_usina.Sem_Monitoramento | df_usina.Homologar_Marcos | df_usina.Revisar_IO)) | df_usina.Em_teste


df_usina['Selecionado_BIU'] = df_usina[list_casos].any(axis=1)

In [65]:
list_criterios = ['criterio_novo',"criterionovopmo"]

df_usina_criterio = df_ug.loc[df_ug.groupby("IdeUsinaOutorga").criterionovopmo.idxmax()][["IdeUsinaOutorga","FASE",'flagOPTeste30dias'] + list_criterios]

################################ dscjustificativaregranova ################################
list_condicoes = [
    (df_usina_criterio['criterionovopmo'] == 9),
    (df_usina_criterio['criterionovopmo'] == 8),
    (df_usina_criterio['criterionovopmo'] == 0.2),
    (df_usina_criterio['criterionovopmo'] == 0.1) | (df_usina_criterio['criterionovopmo'] == 0.3),
    (df_usina_criterio['criterionovopmo'] == 1),
    df_usina_criterio['criterionovopmo']==2,
    df_usina_criterio['criterionovopmo']==3,
    (df_usina_criterio.criterionovopmo== 4),
    (df_usina_criterio.criterionovopmo== 5),
    (df_usina_criterio.criterionovopmo== 6),
    (df_usina_criterio.criterionovopmo== 7),
]

list_values = [
    "Analisar justificativa: Revogação da outorga em avaliação ou Demandas judiciais ou Inviabilidade da implantação da usina",
    "Paralisação de obras",
    "Analisar justificativa: Sem Licença de Operação (LO) ou Alterações de características técnicas ou Estágio das obras da conexão associada ou Demandas judiciais ou Paralisação de obras",
    "Estágio das obras da usina",
    "Compromisso de venda de energia - PPA",
    "Acesso a rede contratado - CUST/CUSD",
    "Acesso a rede não contratado - CUST/CUSD",
    "Sem Licença de Instalação - LI",
    "Sem Licença de Instalação - LI",
    "Nenhuma licença ambiental válida",
    "Sem RAPEEL"
]

df_usina_criterio['dscjustificativaregranova'] = np.select(list_condicoes,list_values)

################################ dsccriterionovo ################################
list_condicoes = [
    ((df_usina_criterio['criterio_novo'] == 9)),
    ((df_usina_criterio['criterio_novo'] == 0) & (df_usina_criterio['FASE'] != "OT")),
    ((df_usina_criterio['criterio_novo'] == 0) & (df_usina_criterio['FASE'] == "OT") & (df_usina_criterio['flagOPTeste30dias'] == 1)),
    ((df_usina_criterio['criterio_novo'] == 0) & (df_usina_criterio['FASE'] == "OT")),
    ((df_usina_criterio['criterio_novo'] == 1)),
    ((df_usina_criterio['criterio_novo'] == 2)),
    ((df_usina_criterio['criterio_novo'] == 3)),
    ((df_usina_criterio['criterio_novo'] == 4)),
    ((df_usina_criterio['criterio_novo'] == 5)),
    ((df_usina_criterio['criterio_novo'] == 7)),
    ((df_usina_criterio['criterio_novo'] == 6)),
    ((df_usina_criterio['criterio_novo'] == 8))
]

list_values = [
    "Usina Viabilidade Baixa análise da fiscalização",
    "Usina em obras = Previsão OC maior entre data calculada e data RAPEEL",
    "Usina em Teste há mais de 30 dias = Previsão OC próximos 60 dias",
    "Usina em Teste = Previsão OC conforme data calculada",
    "Usina sem obras com PPA = Previsão OC maior entre data calculada, data RAPEEL e data compromisso",
    "Usina sem obras com CUST = Previsão OC maior entre data calculada, data RAPEEL e data compromisso",
    "Usina sem obras, sem cust, sem PPA e com LI  = Previsão OC handicap de 5 anos",
    "Usina sem obras, com cust, sem PPA e sem LI  = Previsão OC handicap de 5 anos",
    "Usina sem obras, sem cust, sem PPA e sem LI  = Previsão OC handicap de 5 anos",
    "Usina sem obras, sem PPA e sem RAPEEL  = Previsão OC handicap de 5 anos",
    "Usina sem LP = Previsão OC sem previsão","Usina obras paralisadas = Previsão OC maior entre data RAPEEL e data compromisso"
]
df_usina_criterio['dsccriterionovo'] = np.select(list_condicoes,list_values)
list_justificativas = ['dsccriterionovo','dscjustificativaregranova']

In [66]:
df_usina = pd.merge(df_usina,
df_usina_criterio[['IdeUsinaOutorga','criterionovopmo'] + list_justificativas],
on = "IdeUsinaOutorga", how='left')

In [67]:
list_casos = ['Caso_I','Caso_II_a','Caso_II_b','Caso_III','Selecionado_BIU']
df_ug = pd.merge(df_usina[['IdeUsinaOutorga','dscjustificativaregranova','dsccriterionovo','DatMonitoramento','DscJustificativaPrevisao'] + list_casos],df_ug, on = "IdeUsinaOutorga",how='left')

In [68]:
# Renomeia colunas para exportação
dic = {
    'NumOperacaoUg':'NumUgUsina',
    'regranovapmo': 'PrevisaoOC_regra',
    'dscjustificativaregranova' : 'Justificativadaprevisao_new',
    'criterionovopmo' : 'CriterioPrevisao',
    'dsccriterionovo':'DscCriterioPrevisao',
    'Previsao_OC':'CalculoPrevisorOC',
    'Dat_OC_obrigacao':'OC_Obrigacao',
    'DatPrevistaComercial':'PrevisaoOC_rapeel_max', 
    'DscJustificativaPrevisao':'DscJustificativaPrevisaoAtual',
    'DatPrevisaoIniciobra':'prev_IO_SFG',
    'prev_IO':'prev_IO_rapeel'
}   


df_ug = df_ug.rename(columns=dic)
df_usina = df_usina.rename(columns=dic)

In [69]:
""" ### include_on_script ###
print('BIU gerado.')
return (df_usina,df_ug)
"""
### end_function ###

" ### include_on_script ###\nprint('BIU gerado.')\nreturn (df_usina,df_ug)\n"

In [70]:
#### start_function ### function_name = export_biu_files
### start_function_vars ###
### start_funcion_defined_vars ###
### end_funcion_defined_vars ###
biu_path = biu_path
casos = ['I','I_a','BIU']

""" ### include_on_script ###
df_usina = df_usina
df_ug = df_ug
"""
### end_function_vars ###

if 'I' in casos:
    # Caso I
    file_name = os.path.join(biu_path,f"Caso_I.xlsx")
    df_ug[df_ug.Caso_I][['IdeUsinaOutorga','DatInicioObraOutorgado','prev_IO_rapeel','prev_IO_SFG','DatMonitoramento','DthEnvio']].to_excel(file_name)

# Caso II_a
if 'II_a' in casos:
    file_name = os.path.join(biu_path,f"Caso_II_a.xlsx")
    df_usina[df_usina.Caso_II_a][['IdeUsinaOutorga','DatInicioObraOutorgado','prev_IO_rapeel','prev_IO_SFG','DatMonitoramento','DthEnvio']].to_excel(file_name)
if 'BIU' in casos:
    # Caso II-b e III
    file_name = os.path.join(biu_path,f"BIU.xlsx")
    df_usina[df_usina.Caso_II_b | df_usina.Caso_III].to_excel(file_name)
### end_function ###

## Análise de casos BIU

In [ ]:
list_criterios_BIU = ['Manual', 'Revisar_IO','Homologar_Marcos', 'Em_teste','Sem_Monitoramento']
casos = df_usina[list_criterios_BIU +  list_casos].value_counts().reset_index().sort_values(by='Caso_III')
casos
mask_casos_selecionados = casos[list_casos].any(axis=1)
casos[mask_casos_selecionados]

In [ ]:
df_usina.loc[df_usina.Selecionado_BIU,list_casos+ ['Manual','Homologar_Marcos']].value_counts().reset_index()

In [ ]:
#df_usina[df_usina.Homologar_Marcos & (~ df_usina.Sem_Monitoramento)][['IdeUsinaOutorga','NomUsina','Homologar_Marcos',"Sem_Monitoramento",'DatMonitoramento','DthEnvio','Dsc_Marcos_a_Homologar']].to_excel("Usinas_sem_fotos.xlsx",index=False)

In [ ]:
raise

# Testes

## Junta dados do SAS e Python

In [ ]:
criterios_da_previsao_sas = load_pickle('./criterios_da_previsao_sas.pickle')
tabela_criterio_sas = load_pickle('./tabela_criterio_sas.pickle')
criterios_da_previsao_sas = pd.merge(criterios_da_previsao_sas,tabela_criterio_sas,on="IdeUsinaOutorga").rename(columns={'NumUgUsina':'NumOperacaoUg'})[["IdeUsinaOutorga","NumOperacaoUg",'CriterioNovoPMO','DscJustificativaRegranova','DscCriterioNovo','CalculoPrevisorOC_mx',"FASE"]]

criterios_da_previsao_sas['SAS'] = True
criterios_da_previsao_sas.CalculoPrevisorOC_mx = criterios_da_previsao_sas.CalculoPrevisorOC_mx.dt.normalize()
df_ug["PY"] = True


#criterios_da_previsao_sas.DscJustificativaPMO = criterios_da_previsao_sas.DscJustificativaPMO.str.replace('_x0020_',' ')
#criterios_da_previsao_sas.DscJustificativaPMO = criterios_da_previsao_sas.DscJustificativaPMO.str.replace('_x002F_','/')

criterios_da_previsao_sas.DscJustificativaRegranova = criterios_da_previsao_sas.DscJustificativaRegranova.str.replace('_x0020_',' ')
criterios_da_previsao_sas.DscJustificativaRegranova = criterios_da_previsao_sas.DscJustificativaRegranova.str.replace('_x002F_','/')
criterios_da_previsao_sas.DscJustificativaRegranova = criterios_da_previsao_sas.DscJustificativaRegranova.str.replace('_x0028_','(')
criterios_da_previsao_sas.DscJustificativaRegranova = criterios_da_previsao_sas.DscJustificativaRegranova.str.replace('_x0029_',')')

criterios_da_previsao_sas.DscCriterioNovo = criterios_da_previsao_sas.DscCriterioNovo.str.replace('_x0020_',' ')
criterios_da_previsao_sas.DscCriterioNovo = criterios_da_previsao_sas.DscCriterioNovo.str.replace('_x002F_','/')
criterios_da_previsao_sas.DscCriterioNovo = criterios_da_previsao_sas.DscCriterioNovo.str.replace('_x0028_','(')
criterios_da_previsao_sas.DscCriterioNovo = criterios_da_previsao_sas.DscCriterioNovo.str.replace('_x0029_',')')

criterios_da_previsao_sas.DscCriterioNovo = criterios_da_previsao_sas.DscCriterioNovo.str.replace('_x003D_','=')
criterios_da_previsao_sas.DscCriterioNovo = criterios_da_previsao_sas.DscCriterioNovo.str.replace('_x002C_', ',')

In [ ]:
criterios_da_previsao_sas_gb = criterios_da_previsao_sas.groupby("IdeUsinaOutorga")[['CriterioNovoPMO','DscCriterioNovo','DscJustificativaRegranova']].nunique()
usinas_dois_criterios = criterios_da_previsao_sas_gb[criterios_da_previsao_sas_gb.CriterioNovoPMO>1].index

In [ ]:
criterios_da_previsao_sas_usina = criterios_da_previsao_sas[['IdeUsinaOutorga','CriterioNovoPMO','DscJustificativaRegranova','DscCriterioNovo']].drop_duplicates()

In [ ]:
merge_usina  = pd.merge(df_usina_criterio[['IdeUsinaOutorga','criterionovopmo','dscjustificativaregranova','dsccriterionovo']],
criterios_da_previsao_sas_usina,on=['IdeUsinaOutorga'],suffixes=('_py','_sas'),how='outer')

In [ ]:
criterios_da_previsao_sas_ug = criterios_da_previsao_sas[list_id_ug + ['CalculoPrevisorOC_mx','FASE']]

In [ ]:
merge_ug = pd.merge(df_ug[list_id_ug + ['FASE','Previsao_OC']],criterios_da_previsao_sas_ug,on=['IdeUsinaOutorga','NumOperacaoUg'],suffixes=('_py','_sas'),how='outer')
merge_ug['OC_diff'] = merge_ug.Previsao_OC - merge_ug.CalculoPrevisorOC_mx

## Teste previsão

In [ ]:
merge_ug.sort_values(by='OC_diff',ascending=False).head(20)

## Teste critérios

### FASE ok

In [ ]:
merge_ug[merge_ug.FASE_py != merge_ug.FASE_sas]

### criterionovopmo OK

In [ ]:
merge_usina[(merge_usina.CriterioNovoPMO != merge_usina.criterionovopmo) & (~merge_usina.IdeUsinaOutorga.isin(usinas_dois_criterios))]

In [ ]:
merge_usina[(merge_usina.IdeUsinaOutorga.isin(usinas_dois_criterios))].sample(6)

### dscjustificativaregranova OK

In [ ]:
merge_usina[(merge_usina.dscjustificativaregranova != merge_usina.DscJustificativaRegranova) & (~merge_usina.IdeUsinaOutorga.isin(usinas_dois_criterios))]


In [ ]:
merge_usina[(merge_usina.dscjustificativaregranova != merge_usina.DscJustificativaRegranova)]

### dsccriterionovo OK

In [ ]:
merge_usina[(merge_usina.dsccriterionovo != merge_usina.DscCriterioNovo) & (~merge_usina.IdeUsinaOutorga.isin(usinas_dois_criterios))]